In [26]:
import seaborn as sns 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 
import random
from pprint import pprint

In [27]:
df = sns.load_dataset('iris')
df = df.rename(columns = {"species":"label"})
df.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   label         150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [29]:
# train Test Split 
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size*len(df))
    
    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k = test_size)
    test_df = df.loc[test_indices,:]
    train_df = df.drop(test_indices)

    return train_df, test_df

In [30]:
# data purity check 
def check_purity(data):

    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    return False

In [31]:
# classify

def classify_data(data):
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts = True)
    index = counts_unique_classes.argmax()
    classification = unique_classes[index]

    return classification 

In [32]:
def get_potential_splits(data):
    potential_splits = {}
    _, n_columns = data.shape

    for column_index in range(n_columns - 1):
        
        values = data[:, column_index]
        unique_values = np.unique(values)

        
        types_of_feature = FEATURE_TYPES[column_index]
        if types_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]

                    potential_split = (current_value + previous_value)/2
                    potential_splits[column_index].append(potential_split)

        
        else:
            potential_splits[column_index] = unique_values

    return potential_splits

In [33]:
def split_data(data, split_column, split_value):
    split_column_values = data[:, split_column] 
    split_column_values <= split_value

    types_of_feature = FEATURE_TYPES[split_column]
    if types_of_feature == "Continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]
    
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]


    return data_below, data_above

In [34]:
# lowest overall entropy

def calculate_entropy(data):
    label_column = data[:,-1]
    _, counts = np.unique(label_column, return_counts = True)
    probabilities = counts/counts.sum()
    entropy = sum(probabilities*-np.log2(probabilities))

    return entropy 

In [35]:
def calculate_overall_entropy(data_below, data_above):

    n_data_points = len(data_below) + len(data_above)
    p_data_below = len(data_below)/n_data_points
    p_data_above = len(data_above)/n_data_points

    overall_entropy = (p_data_below * calculate_entropy(data_below) + 
                       p_data_above * calculate_entropy(data_above))

    return overall_entropy

In [36]:
def determine_best_split(data, potential_splits):
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column = column_index, split_value = value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value

    

    return best_split_column, best_split_value

Representation of the Decision Tree (using dictionary) 

`sub_tree = {question : [yes_answer, no_answer]}`

In [37]:
example_tree = {"petal_width":["setosa",
                               {"petal_width <= 1.65" : [{"petal_length <= 4.9" : ["versicolor",
                                                                                   "virginica"]},
                                                         "virginica"]}]}

In [38]:
def determine_type_of_features(df):
    feature_types = []
    n_unique_values_threshold = 15

    for column in df.columns: 
        unique_values = df[column].unique()
        example_value = unique_values[0]

        if (isinstance(example_value, str)) or (len(unique_values)<= n_unique_values_threshold):
            feature_types.append('Categorical')

        else:
            feature_types.append('Continuous')


    return feature_types

In [39]:
### Algorithm
def decision_tree_algorithm(df, counter = 0, min_samples = 2, max_depth = 5):

    # data preparations
    if counter == 0:
        global COLUMN_HEADERS
        global FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_features(df)
        data = df.values
    
    else:
        data = df


    # base case 
    if (check_purity(data) or len(data) < min_samples or counter == max_depth):
        classification = classify_data(data)
        return classification 
    
    # recursive part 
    else:
        counter += 1

        # helper functions
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data,split_column, split_value)


        # instantiate subtree 
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        
        if type_of_feature == "Continuous":
            question = f"{feature_name} <= {split_value}"

        else:
            question = f"{feature_name} = {split_value}"

        
        subtree = {question : []}

        # find answers (recursion)

        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)

        if yes_answer == no_answer:
            subtree = yes_answer
        
        else:
            subtree[question].append(yes_answer)
            subtree[question].append(no_answer)

        return subtree 


    

In [40]:
# Classification 
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split()

    # ask question
    if comparison_operator == "<=": 
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]


    # base case 
    if not isinstance(answer, dict):
        return answer
    else:
        return classify_example(example, answer) 


In [41]:
def calculate_accuracy(df, tree):
    df["classification"] = df.apply(classify_example, axis = 1, args = (tree,))
    df["classification_correct"] = df.classification == df.label
    accuracy = df.classification_correct.mean()

    return accuracy

In [42]:
train_df, test_df = train_test_split(df, test_size= 0.2)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

pprint(tree)
print(f"Accuracy: {accuracy}")

{'petal_width <= 0.6': ['setosa',
                        {'petal_width <= 1.6': [{'petal_length <= 4.9': ['versicolor',
                                                                         {'petal_width <= 1.5': ['virginica',
                                                                                                 {'petal_length <= 5.1': ['versicolor',
                                                                                                                          'virginica']}]}]},
                                                {'petal_length <= 4.8': [{'sepal_width <= 3.0': ['virginica',
                                                                                                 'versicolor']},
                                                                         'virginica']}]}]}
Accuracy: 0.9666666666666667


In [43]:
test_df

,sepal_length,sepal_width,petal_length,petal_width,label,classification,classification_correct
54,6.5,2.8,4.6,1.5,versicolor,versicolor,True
52,6.9,3.1,4.9,1.5,versicolor,versicolor,True
14,5.8,4.0,1.2,0.2,setosa,setosa,True
117,7.7,3.8,6.7,2.2,virginica,virginica,True
95,5.7,3.0,4.2,1.2,versicolor,versicolor,True
60,5.0,2.0,3.5,1.0,versicolor,versicolor,True
74,6.4,2.9,4.3,1.3,versicolor,versicolor,True
65,6.7,3.1,4.4,1.4,versicolor,versicolor,True
82,5.8,2.7,3.9,1.2,versicolor,versicolor,True
135,7.7,3.0,6.1,2.3,virginica,virginica,True


In [44]:
# Titanic Data Set
df = pd.read_csv("Data sets/Titanic-Dataset.csv")
df["label"] = df.Survived
df = df.drop(["Survived", "PassengerId", "Name", "Ticket", "Cabin"], axis = 1)


# handling missing values 
median_age = df.Age.median()
mode_embarked = df.Embarked.mode()[0]
df = df.fillna({"Age":median_age, "Embarked": mode_embarked})

In [ ]:
df.info()

In [45]:
train_df, test_df = train_test_split(df, test_size= 0.2)
tree = decision_tree_algorithm(train_df)
accuracy = calculate_accuracy(test_df, tree)

pprint(tree)
print(f"Accuracy: {accuracy}")

{'Sex = male': [{'Fare <= 26.25': [{'Age <= 12.0': [{'SibSp = 1': [1,
                                                                   {'Fare <= 20.525': [1,
                                                                                       0]}]},
                                                    {'Embarked = C': [{'Fare <= 15.05': [0,
                                                                                         1]},
                                                                      0]}]},
                                   {'Fare <= 26.3875': [1,
                                                        {'SibSp = 4': [0,
                                                                       {'Fare <= 263.0': [0,
                                                                                          1]}]}]}]},
                {'Pclass = 3': [{'Fare <= 24.15': [{'Embarked = S': [{'Fare <= 7.4958': [1,
                                                                

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  891 non-null    object 
 7   label     891 non-null    int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB
